In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In the paper, we assume each player draws their true parameters i.i.d. (independent and identically distributed) $(\theta_j, \epsilon^2_j) \sim \Theta$, for some joint distribution $\Theta$. $\epsilon^2_j$ represents the amount of noise present in the sampling process for a given player. For linear regression, $\theta_j$ is a $D$-dimensional vector representing the coefficients on the true classification function, which is also assumed to be linear. This vector is what each agent wishes to learn. 

In this notebook, we analyze a one of the simplified cases, where the distribution of input values $\mathcal{X}_{j}$ is a $D$-dimensional multivariate normal distribution with 0 mean, and where the coefficients $\theta_d$ for each dimension are drawn independently of each other. In this case, the true expected MSE each player experiences depends only on $n$, the number of samples they have, $\mu_e = \mathbb{E}[\epsilon^2]$, and $\sigma_d^2 = Var(\theta_d)$, a $D$ length vector of variances.   

For this code, we need a $\Theta$ to draw from. For simplicity, we assume $\theta_d \ d \in [D], \epsilon^2$ are independent of each other. In the paper, we use $\mu_e = 10$ as a common parameter, which we use below. We've selected $D=2$ and $\theta_1 = 1, \theta_2 = 2$ for simplicity, which are the values below. Note that the results do not depend on the distributions themselves or other values besides $\sigma_1^2, \sigma_2^2, \mu_e$.

In [2]:
means_dist_1 = stats.norm(loc = 0, scale = 1)
means_dist_2 = stats.norm(loc = 3, scale = 2)
variance_dist = stats.beta(a=8, b=2, scale = 50/4)
print([means_dist_1.var(), means_dist_2.var()])
print(variance_dist.mean())

[1.0, 4.0]
10.0


calculate_regression() calculates the exact expected MSE a set of players experiences. For reference, the equations it is using are provided below: 

Local learning: $$\frac{\mu_e}{n - D - 1}$$

Uniform federation: 
$$\mu_e\sum_{i=1}^{N}\frac{n_i^2}{N^2}\frac{D}{n_i - D - 1} + \frac{\sum_{i\ne j}n_i^2 + (N - n_j)^2}{N^2}\sum_{d=1}^{D}\mathbb{E}_{x \sim \mathcal{X}_j}[(x^d)^2] \cdot \sigma^2_d$$
where $N = \sum_{i=1}^{M}n_i$, and $M$ is the number of players. 

Coarse-grained federation: 
$$\mu_e \cdot (1-w)^2 \cdot \sum_{i=1}^{M}\frac{n_i^2}{N^2}\frac{D}{n_i - D - 1}+ \mu_e \cdot \left(w^2 + 2 \cdot \frac{(1-w) \cdot w \cdot n_j}{N}\right) \cdot \frac{D}{n_i - D - 1} + (1-w)^2 \cdot \frac{\sum_{i\ne j}n_i^2 + (N - n_j)^2}{N^2}\sum_{d=1}^{D}\mathbb{E}_{x \sim \mathcal{X}_j}[(x^d)^2] \cdot \sigma^2_d$$
where $w$ is a parameter. 

Fine-grained federation: 

$$\mu_e\sum_{i=1}^{M}v_{ji}^2\cdot \frac{D}{n_i - D - 1} + \left(\sum_{i\ne j}v_{ji}^2 + \left(\sum_{i\ne j}v_{ji}\right)^2\right)\cdot \sum_{d=1}^{D}\mathbb{E}_{x \sim \mathcal{X}_j}[(x^d)^2] \cdot \sigma^2_d$$

In [3]:
def calculate_regression(var_vec = [means_dist_1.var(), means_dist_2.var()], mue = variance_dist.mean(), 
                         n_list = [10, 20, 30], w_list = [0.2, 0.4, 0.6], 
                         v_mat = [[0.1, 0.6, 0.3], [0.2, 0.8, 0.0], [0.3, 0.5, 0.2]], 
                         x_vec = [1.0, 1.0]):
    '''
    Calculate exact error for linear regression, in special case where x-distribution is 0-mean multivariate 
    normal. Assumes the parameters have 0 correlation (parameter in 1st dimension is independent of parameter
    in 2nd dimension, for example). 

    Args:
        var_vec: a list with variance of true parameter values
        mue: mean of true error distribution. 
        n_list: a list of length M (number of players) with the number of samples each has. 
        w_list: a list of w-weights each player uses (in [0, 1]) for coarse-grained federation. 
        v_mat: a matrix (list of lists) of weights each player uses in fine-grained federation: the rows sum up 
               to 1.
        x_vec: a list of E[X_d^2] for d in dimension D for input distribution of X. 
    Returns:
        dataframe with average error for each player, for: local, uniform, coarse-grained, and fine-grained 
        federation.  
    '''
    # dataframe for storing error
    player_error = pd.DataFrame(data = 0.0, index = ['local', 'uniform', 'coarse', 'fine'], 
                                columns = range(len(n_list)))
    N = sum(n_list)
    D = len(var_vec)
    n_vec = pd.DataFrame(n_list)
    
    # for each player, calculate their true error 
    for i in range(len(w_list)):
        w = w_list[i]
        n = n_list[i]
        v_vec = pd.DataFrame(v_mat[i])
        
        # local
        player_error.loc['local'][i] = mue * D/(n - D - 1)
        
        # uniform
        var_prod = pd.DataFrame(x_vec).T.dot(pd.DataFrame(var_vec))[0][0]
        
        player_error.loc['uniform'][i] = ((mue * D * (n_vec**2).T.dot(1/(n_vec- D -1))/(N**2))[0][0] + 
                                          var_prod*((n_vec**2).sum() - n**2 + (N- n)**2)[0]/(N**2))
        
        # coarse-grained
        player_error.loc['coarse'][i] = ((mue*D *(1-w)**2 * (n_vec**2).T.dot(1/(n_vec- D -1))/(N**2))[0][0] + 
                                         mue*D*(w**2 + 2 * (1-w) * w*n/N)/(n-D-1) + 
                                         (1-w)**2*var_prod*((n_vec**2).sum() - n**2 + (N- n)**2)[0]/(N**2))    
      
        # fine-grained
        player_error.loc['fine'][i] = (mue * D * (v_vec**2).T.dot(1/(pd.DataFrame(n_list)- D -1))[0][0] + 
                                       var_prod * ((v_vec**2).sum() - v_vec[0][i]**2 + (1 - v_vec[0][i])**2)[0])
        
    return player_error

This returns a dataframe: each column represents a different player and each row represents the error that player experiences under different local or federation strategies. 

In [4]:
calculate_regression()

,0,1,2
local,2.857143,1.176471,0.740741
uniform,5.673047,4.006380,2.339714
coarse,3.897417,1.818768,0.818799
fine,6.818768,1.267227,5.480890


simulate_regression() simulates the process described above: players draw true mean and variance parameters $\theta_d, \epsilon^2$, then $x$ vectors $X_j \sim \mathcal{X}_j$, then for each $x$ draw a noisy measure
$Y_j \sim D_j(X_j^T \theta_j, \epsilon^2_j)$ $Y \sim D_j(\theta, \epsilon^2)$ and calculate empirical means using local learning or some variant of federation. Results should match calculate_regression() closely, if the same parameters are used and if sufficient simulations are used. 

In [5]:
def simulate_regression(params_dists = [means_dist_1, means_dist_2], 
                        err_dist = variance_dist, draws_dist = stats.norm, 
                        n_list = [10, 20, 30], w_list = [0.2, 0.4, 0.6], 
                        v_mat = [[0.1, 0.6, 0.3], [0.2, 0.8, 0.0], [0.3, 0.5, 0.2]], 
                        world_nrun = 100, sample_nrun = 1, test_nrun = 10,
                        x_cov = [[1.0, 0.0], [0.0, 1.0]]):
    
    '''
    Simulate regression. 
    

    Args:
        params_dists: length-D list of distributions to draw parameters from.  
        err_dist: distribution to draw true error parameters from (err = epsilon^2) (scalar)
        draws_dist: distribution each player draws from: with mean*X as mean and variance epsilon^2. 
        n_list: a list of length M (number of players) with the number of samples each has. 
        w_list: a list of w-weights each player uses for coarse-grained federation.  
        v_mat: a matrix (list of lists) of weights each player uses in fine-grained federation: the rows sum up 
               to 1.
        world_nrun: number of times where means and errors are re-drawn
        sample_nrun: for each worldrun, number of times samples are re-drawn
        test_nrun: for each sample_nrun, number of points we use to calculate expected test error
        x_cov: matrix representing covariance of input x distribution
        
    Returns:
        dataframe with average error for each player, for local, uniform, coarse-grained, and fine-grained 
        federation.  
    '''
    M = len(w_list)
    n_list_pd = pd.DataFrame(n_list)
    w_list_pd = pd.DataFrame(w_list)
    v_mat_pd = pd.DataFrame(v_mat)
    
    D = len(x_cov)
    mean_x = np.array([0] * D)
    x_dist = stats.multivariate_normal(mean = mean_x, cov = x_cov)
    
    # dataframe for storing error
    player_error = pd.DataFrame(data = 0, index = ['local', 'uniform', 'coarse', 'fine'], 
                                columns = range(len(n_list)))
    
    
    for wn in range(world_nrun):
        # draw means and errors
        means = pd.DataFrame([dist.rvs(M) for dist in params_dists]).T 
        errors = err_dist.rvs(M)    
        
        for sn in range(sample_nrun): 
            
            # draw samples for each player, calculate local estimates
            local_est = pd.DataFrame(0, columns = range(D), index = range(M))
            for i in range(M):
                # draw X values
                X = pd.DataFrame(x_dist.rvs(n_list[i]))
                
                # draw Y values nosily
                Y = [draws_dist(X.dot(means.iloc[i])[j], np.sqrt(errors[i])).rvs(1)[0] for j in range(n_list[i])]
                
                # calculate local estimates through OLS
                to_invert = X.T.dot(X)
                df_inv = pd.DataFrame(np.linalg.pinv(to_invert.values), to_invert.columns, to_invert.index)
                local_est.iloc[i] = df_inv.dot(X.T).dot(Y)
            
            # calculate federated estimates
            uniform_est = local_est.T.dot(n_list_pd)/sum(n_list)
            coarse_est = (local_est * w_list_pd.values + 
                          (1-w_list_pd).values * pd.concat([uniform_est.T]*M, ignore_index=True)) 
            fine_est = local_est.T.dot(v_mat_pd.T).T
            
            # calculate expected MSE
            X = pd.DataFrame(x_dist.rvs(test_nrun)) # draw test input data
            if test_nrun ==1:
                X = X.T
                
            player_error.loc['local'] += ((X.dot(means.T) - X.dot(local_est.T))**2).sum()
            player_error.loc['uniform'] += ((X.dot(means.T) - 
                                 X.dot(pd.concat([uniform_est.T] * M, ignore_index=True).T))**2).sum()
            player_error.loc['coarse'] += ((X.dot(means.T) - X.dot(coarse_est.T))**2).sum()
            player_error.loc['fine'] += ((X.dot(means.T) - X.dot(fine_est.T))**2).sum()
    
    player_error = player_error/(world_nrun * sample_nrun * test_nrun)
    
    return player_error

Compare simulations to exact values. 

In [6]:
simulate_regression(world_nrun = 1000)

,0,1,2
local,2.703608,1.249858,0.720794
uniform,5.384239,4.098849,2.304755
coarse,3.693648,1.854036,0.796649
fine,6.619304,1.325227,5.387945


In [7]:
calculate_regression()

,0,1,2
local,2.857143,1.176471,0.740741
uniform,5.673047,4.006380,2.339714
coarse,3.897417,1.818768,0.818799
fine,6.818768,1.267227,5.480890
